# Goodreads Database

## Summary
<b>Goals</b>:
- create a database with 5 tables:
    - book clubs
    - books
    - book-bookclub pairs
    - genres
    - book-genre pairs  
    
<b>Source data</b>: 
- csv file for GoodReads created in Phase 1 of the project (link)  

<b>Implementation:</b>  
For this purposes there has been used Amazon RDS instance with MySQL rdbms. We need to designe the schema, implement this schema on the existing instance and then populate the created tables with our data.  

## DB Schema

In [4]:
from IPython.display import Image
from IPython.core.display import HTML 
HTML("<h3>ER-diagram</h3>")

In [5]:
Image(url= "er_goodreads.png")

### Book clubs creation script

In [6]:
book_club_sql = """
CREATE TABLE GoodReadsDB.book_club (
	book_club_id INT NOT NULL AUTO_INCREMENT,
	club_name varchar(255) NOT NULL,  
	CONSTRAINT gr_books_PK PRIMARY KEY (book_club_id)
)  
ENGINE=InnoDB  
DEFAULT CHARSET=utf8  
COLLATE=utf8_general_ci;
"""

### Books table creation script

In [7]:
books_table_sql ="""
CREATE TABLE GoodReadsDB.book (
	book_id BIGINT NOT NULL,
	title varchar(255) NOT NULL,  
	reviews_cnt INT NULL,  
	ratings_cnt INT NULL,  
	pub_year INT NULL,  
	avg_rating FLOAT NULL,  
	alt_avg_rating FLOAT NULL,  
	author_1_name varchar(100) NULL,  
	author_1_avg_rating FLOAT NULL,  
	CONSTRAINT gr_books_PK PRIMARY KEY (book_id)
)  
ENGINE=InnoDB  
DEFAULT CHARSET=utf8  
COLLATE=utf8_general_ci;
"""

### BookClub-Book table creation script

In [8]:
bookclub_book_table_sql ="""
CREATE TABLE GoodReadsDB.book_club_book (
	id BIGINT NOT NULL AUTO_INCREMENT,
	book_id BIGINT NOT NULL,
	book_club_id INT NOT NULL,
	CONSTRAINT gr_books_PK PRIMARY KEY (id),
	CONSTRAINT book_club_book_book_FK FOREIGN KEY (book_id) REFERENCES GoodReadsDB.book(book_id),    
	CONSTRAINT book_club_book_book_club_FK FOREIGN KEY (book_club_id) REFERENCES GoodReadsDB.book_club(book_club_id)        
)  
ENGINE=InnoDB  
DEFAULT CHARSET=utf8  
COLLATE=utf8_general_ci;
"""

### Genres table creation script

In [9]:
genres_table_sql = """
CREATE TABLE GoodReadsDB.genre (
	genre_id INT NOT NULL,
	name varchar(100) NULL,
	CONSTRAINT gr_genres_PK PRIMARY KEY (genre_id)
)
ENGINE=InnoDB
DEFAULT CHARSET=utf8
COLLATE=utf8_general_ci;
"""

### Genre-book table (many-to-many relationship)

In [10]:
genre_book_sql = """
CREATE TABLE GoodReadsDB.genre_book (
	id BIGINT NOT NULL AUTO_INCREMENT,
	book_id BIGINT NOT NULL,
	genre_id INT NOT NULL,
	CONSTRAINT genre_book_PK PRIMARY KEY (id),
	CONSTRAINT genre_book_book_FK FOREIGN KEY (book_id) REFERENCES GoodReadsDB.book(book_id),
	CONSTRAINT genre_book_genre_FK FOREIGN KEY (genre_id) REFERENCES GoodReadsDB.genre(genre_id)
)
ENGINE=InnoDB
DEFAULT CHARSET=utf8
COLLATE=utf8_general_ci;
"""

## Step 1: Book clubs table

For this step we only need to create a .csv with one single column with book club names. We can the populate the table with this data as follows:

In [149]:
book_clubs = pd.DataFrame({'club_name': ["Bertelsmann Data Science book readers",
                                          "Gone with a Book", 
                                          "Pop Sugar's Annual Ultimate Reading Challenge",
                                          "Reading with Style"]})

In [150]:
book_clubs

,club_name
0,Bertelsmann Data Science book readers
1,Gone with a Book
2,Pop Sugar's Annual Ultimate Reading Challenge
3,Reading with Style


In [11]:
book_club_populate_script_sql = """
LOAD DATA LOCAL INFILE '/Users/anamakarevich/UdacityProjects/book_clubs.csv'
INTO TABLE book_club 
FIELDS TERMINATED BY ','
    ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 ROWS
(club_name)
"""

Since we're using autoincrement for this table for the id field, the book club names will be assigned ids from 1 to 4 (not 0 to 3 as shown in the index). We will need that information in step 2.

## Step 2: Book table

### Preparing the data

In [135]:
import pandas as pd

In [235]:
# list the names of the source files in the same order we listed the books clubs
file_names = ['gr_books_v4.csv', 'gr_books_gone_with_book_v4.csv',
              'gr_books_pop_sugar_challenge_v4.csv',
              'gr_books_reading_with_style_v4.csv']
# list the new names for the cleaned data
new_file_names = ['bert_books_clean.csv', 'gone_with_the_book_books_clean.csv',
                 'pop_sugar_challenge_books_clean.csv', 'reading_with_style_books_clean.csv']
book_clubs_data = [pd.read_csv(x) for x in file_names]
number_of_clubs = len(book_clubs_data)

First we need to unify the book ids across different book clubs. I.e. the same book can have the same title, but different id depending on edition. Since all the data is about the book, we don't care about edition at all. The idea is to iterate through the data frames and compare them with one another replacing the ids for the duplicate book in the first one with the ids from the second one.  

In [236]:
def check_consistency(dataframes, column_name) -> None:
    """Checks if the dataset is consistent (each title is associated with just one id)
    """
    for i in range(number_of_clubs):
        current_df = dataframes[i]
        current_set = set(current_df[column_name])
        for j in range(number_of_clubs):
            if i!=j:
                other_df = dataframes[j]
                common_titles = current_set.intersection(other_df[column_name])
                if len(common_titles) > 0: 
                    for title in common_titles:
                        titles_equal = (int(current_df[current_df.title==title]['id']) ==
                                        int(other_df[other_df.title==title]['id']))                                        
                        if not titles_equal:
                            print("Data set is inconsistent")
                            return
    print("Data set is consistent")

In [271]:
def unify(df_1, df_2, name, col_names):
    """Replace the id in the first passed dataframe with the id in the secon one for the given name    
    """
    for col_name in col_names:
        # replace first occurence with second
        new_value = int(df_2[df_2.title==name][col_name])
        df_1.loc[df_1.title==name, col_name] = new_value

In [239]:
check_consistency(book_clubs_data, 'title')

Data set is inconsistent


In [272]:
def unify_books(dataframes, column_name):
    for i in range(number_of_clubs):
        current_df = dataframes[i]
        current_set = set(current_df[column_name])
        for j in range(number_of_clubs):
            if i!=j:
                other_df = dataframes[j]
                common_titles = current_set.intersection(other_df[column_name])
                if len(common_titles) > 0: 
                    for title in common_titles:
                        unify(other_df, current_df, title, ('id',
                                                                    'reviews_cnt',
                                                                    'ratings_cnt', 
                                                                    'alt_avg_rating',
                                                                    'avg_rating'
                                                                   ))
    return dataframes

In [273]:
unified_dataset = unify_books(book_clubs_data, 'title')

In [274]:
check_consistency(unified_dataset, 'title')

Data set is consistent


Now we need to assign each book to a book club. Since we know exactly the ids of the books clubs and the dataframes are listed in the same order as the book clubs, this can be simply done with the loop.
We will also select only the columns we need for the proecessing and save it to files. 

In [275]:
unused_columns = ['index', 'author1_role', 'author2_name','author2_role',
                  'author2_avg_rating', 'author3_name', 
                  'author3_role', 'author3_avg_rating']

In [305]:
for i in range(number_of_clubs):    
    current_df = unified_dataset[i]
    selection = list(set(current_df) - set(unused_columns))
    selection.sort()
    current_df['book_club_id'] = i+1
    current_df.to_csv(new_file_names[i],index=False, columns = selection)

In [307]:
clean_dataset = [pd.read_csv(x) for x in new_file_names]

Now we need to extract only the books (excluding the book clubs and genres information)

In [308]:
book_columns = ['id', 'title','reviews_cnt',
               'ratings_cnt', 'pub_year', 'avg_rating',
               'alt_avg_rating',
                'author1_name',
                'author1_avg_rating']

In [326]:
books = pd.concat(clean_dataset, axis=0)[book_columns].drop_duplicates().reset_index(drop=True)

In [315]:
books.to_csv('books_only_final.csv', index = False)

### Populating the table

In [318]:
books_table_populate_sql = """
LOAD DATA LOCAL INFILE '/Users/anamakarevich/UdacityProjects/books_only_final.csv'
INTO TABLE GoodReadsDB.book
FIELDS TERMINATED BY ','
    ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 ROWS
(book_id, title, reviews_cnt, ratings_cnt, pub_year, avg_rating, alt_avg_rating, author_1_name, author_1_avg_rating)
"""

In [333]:
def get_full_dataset():
    dataframes_names = ['bert_books_clean.csv', 'gone_with_the_book_books_clean.csv',
                        'pop_sugar_challenge_books_clean.csv', 'reading_with_style_books_clean.csv']
    clean_dataset = [pd.read_csv(x) for x in dataframes_names]
    all_books = pd.concat(clean_dataset, axis=0)
    return all_books

## Step 3: Book - book club table 

In [393]:
all_books = get_full_dataset()

In [397]:
book_bookclub = all_books[['id','book_club_id']]

In [398]:
book_bookclub.to_csv('book_bookclub.csv', index=False)

In [400]:
bookclub_data_populate_sql = """
LOAD DATA LOCAL INFILE '/Users/anamakarevich/UdacityProjects/book_bookclub.csv'
INTO TABLE GoodReadsDB.book_club_book
FIELDS TERMINATED BY ','
    ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 ROWS
(book_id, book_club_id)
"""

## Step 4: Genres table

### Preparing the data 

For genres we need to extract all the genres present in the table and assign them some ids so that we can reference them in the book_genre table. 
First, we will create a helper function to generate the column names for genres. 

In [330]:
def get_columns(n_genres):
    result = []
    for i in range(1,n_genres+1):
        result.append('Genre_'+str(i))
    return result

In [331]:
# get the columns with genres
genre_columns = get_columns(11)
print(genre_columns)

['Genre_1', 'Genre_2', 'Genre_3', 'Genre_4', 'Genre_5', 'Genre_6', 'Genre_7', 'Genre_8', 'Genre_9', 'Genre_10', 'Genre_11']


Next we actually extract the values of the genres for all the books and create a set of genres.

In [334]:
all_books = get_full_dataset()

In [335]:
# extract just the genres (replacinng None with empty string so that it's easy to remove it later)
set_of_genres = set(all_books[genre_columns].fillna('').values.flatten())
# remove empty strings
set_of_genres.remove('')
print(set_of_genres)

{'thriller', '20th-century', 'science-fiction', 'adult', 'horror', 'humor', 'paranormal', 'american', 'science', 'biography', 'novel', 'gothic', 'classics', 'philosophy', 'dnf', '19th-century', 'young-adult', 'literature', 'women', 'mystery', 'contemporary', 'crime', 'fiction', 'dystopia', 'audio-books', 'non-fiction', 'children', 'politics', 'drama', 'british', 'history', 'translated', 'economics', 'short-stories', 'psychology', 'adventure', 'personal-development', 'france', 'romance', 'christmas-books', 'magic', 'germany', 'chick-lit', 'fantasy', 'wwii'}


We have successfully extracted our genres and ready to create a source file for genres. The last thing we need to do is to assign ids to genres which can be done very easily when creating a data frame.  
Note: we could have created ids automatically in the db, but I decided to try both ways. The last table with use automatic id creation with autoincrement. 

In [336]:
genres_df = pd.DataFrame({'genre_id':range(len(set_of_genres)), 'name': list(set_of_genres)})
# save to .csv
genres_df.to_csv('genres_only.csv', index = False)
genres_df.head()

,genre_id,name
0,0,thriller
1,1,20th-century
2,2,science-fiction
3,3,adult
4,4,horror


### Populating the table 

In [338]:
genres_table_populate_sql = """
LOAD DATA LOCAL INFILE '/Users/anamakarevich/UdacityProjects/genres_only.csv'
INTO TABLE GoodReadsDB.genre
FIELDS TERMINATED BY ','
    ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 ROWS
(genre_id, name)
"""

## Step 3: Create genre-book table 

The last table is the most complicates since we have many to many relationship here which should be implemented with the additional table that matches books ids to genres ids.  
The first thing to do is to create a lookup dictionary for genres and ids so that we can build the required table. 

In [339]:
# create dictionary for genres for fast lookup
genres_dict = dict(zip(genres_df.name, genres_df.genre_id))

In [340]:
print(genres_dict)

{'thriller': 0, '20th-century': 1, 'science-fiction': 2, 'adult': 3, 'horror': 4, 'humor': 5, 'paranormal': 6, 'american': 7, 'science': 8, 'biography': 9, 'novel': 10, 'gothic': 11, 'classics': 12, 'philosophy': 13, 'dnf': 14, '19th-century': 15, 'young-adult': 16, 'literature': 17, 'women': 18, 'mystery': 19, 'contemporary': 20, 'crime': 21, 'fiction': 22, 'dystopia': 23, 'audio-books': 24, 'non-fiction': 25, 'children': 26, 'politics': 27, 'drama': 28, 'british': 29, 'history': 30, 'translated': 31, 'economics': 32, 'short-stories': 33, 'psychology': 34, 'adventure': 35, 'personal-development': 36, 'france': 37, 'romance': 38, 'christmas-books': 39, 'magic': 40, 'germany': 41, 'chick-lit': 42, 'fantasy': 43, 'wwii': 44}


Next we will write a function that will actually match the book ids and genre ids. To do that, we need to iterate though through the rows of the original data frame - we only need two columns: id (book id) and genres which contains a list of genres. We user .values instead of .iterrows() because .values actually provides us with the numpy array which is more efficient for this purpose. And that's why we address the columns by numbers instead of name - each row is just an array in this case. 

In [389]:
def generate_many_to_many_df(df, genres_dict):
    books_ids = []
    genres_ids = []
    for row in df[['id','genres']].values:
        book_id = row[0]
        genres = []
        try:
            genres = row[1].split(',')
        except:
            print("The following row has not been parsed: ")
            print(row[1])
        for genre in genres:
            gen_id = genres_dict.get(genre)
            # we use is non None here since we have 0 as one of the values in the 
            # dictionary which will be treated as None if we don't specify that explicitly
            if gen_id is not None:
                books_ids.append(book_id)
                genres_ids.append(gen_id)
    return pd.DataFrame({'book_id': books_ids, 'genre_id': genres_ids}).drop_duplicates().reset_index(drop=True)
#res_df = generate_many_to_many_df()

In [390]:
book_genre = generate_many_to_many_df(all_books, genres_dict)

The following row has not been parsed: 
nan


Now we can finally save our last data frame to csv

In [391]:
book_genre.to_csv('genre_book.csv', index=False)
book_genre.head()

,book_id,genre_id
0,37424706,25
1,37424706,32
2,37424706,36
3,37424706,34
4,117833,22


### Populating the table 

In [392]:
genre_book_populate_sql = """
LOAD DATA LOCAL INFILE '/Users/anamakarevich/UdacityProjects/genre_book.csv'
INTO TABLE GoodReadsDB.genre_book
FIELDS TERMINATED BY ','
    ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 ROWS
(book_id, genre_id)
"""

## Results

We can check that it all works by joining all tables. For example, let's extract all the genres for the book named "Breakfast at Tiffany's".

In [401]:
test_query = """
SELECT b.title, g.name, bc.club_name, b.avg_rating
FROM book b
JOIN genre_book gb ON b.book_id = gb.book_id
JOIN genre g ON g.genre_id = gb.genre_id
JOIN book_club_book bcb ON bcb.book_id = b.book_id
JOIN book_club bc ON bc.book_club_id = bcb.book_club_id
WHERE b.title = 'Breakfast at Tiffany\'s'
"""

In [402]:
Image(url= "gr_results.png")